# Comparison between Neo4j and Networkx
## Introduction

This notebook aims to compare some algorithms implemented in both Neo4j Graph Data Science and Python's package Networkx.
I used the database from https://github.com/mathbeveridge/asoiaf corresponding to interactions between the characters of the books of the saga Song of Ice and Fire by G.R.R Martin.

I ran all the tests on my PC with the following configuration :
- Intel Core  i5-9300H 2,40 GHz, 4 core, 8 Mo cache memory (turbo 4,10 GHz)
- 12 GB RAM 

## Summary of data
The dataset was made according to the books. Two characters are interacting if they are in a 15-words range in a book. The more they are interacting the more the interactions weight is high. 

Here we consider the 5 first books. There is 2823 relations on 796 nodes which are the characters in the saga.

Here are some stats about the interactions of characters.

|min|max|avg interactions|stdev|
|:-:|:-:|:-:|:-:|
|1|110|4.620294599017998|8.651722878451903|

## How do I compare
Due to the implementation of Neo4j's graphs, I had considered 2 different mesures of time. One is the first iteration of the algorithm and the other is the average time of the following iterations where the Neo4j's cache memory reduces the computation time.

In order to compare networkx and Neo4j, I chose to compare the different implemantion of Betweenness centrality, Page Rank, Label Propagation, BFS and MST.
They are different type of algorithms which are implemented both in Neo4j and Networkx. Much of the algorithms are implemented in only one of them.
I will compare the average time to compute these algorithms with the same graph. 

### The Algorithms 
#### Page Rank
This algorithm aims to compute the impact of a node on the graph. I assume Neo4j and Networkx used the same implemented of this algorithms like in this [paper](http://infolab.stanford.edu/~backrub/google.html).
I configure damping= 0.85 and max iterations=20 in both softwares.

#### Betweenness Centrality
This algorithm returns the node which is in the most path between nodes. I don't really know if the implementations are the same.
I used the default parameters.

#### Label Propagation
This algorithm is a community clustering algorithm which creates community of nodes whose have similary labels.
I think the implementations are the same because the algorithm is clear as we can see [here](https://neo4j.com/docs/graph-data-science/current/algorithms/label-propagation/).
I also used default parameters.

#### BFS
This algorithm is a classic in graph analysis. I assume both implementations are equivalent. 
I limit the depth at 5 nodes and start at the node `Jon Snow`.

#### MST
It is also a classic algorithm. 
Neo4j used Prim's algorithm and Networkx used Kruskal's one. The complexity is similar between the two algorithms.
I used default parameters.



## Creation of the GOT's Graph

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
table=pd.read_csv('./data/data/asoiaf-all-edges.csv')

In [3]:
import networkx as nx
G=nx.Graph(name="Game of Networks")
n=len(table['Source'])
for i in range(n):
    G.add_edge(table['Source'][i],table['Target'][i],weight=table['weight'][i])



## Networkx's tests

### Betweenness Centrality



In [4]:
from time import *

In [5]:
liste=[]
for i in range(100):
    a=time()
    nx.betweenness_centrality(G)
    b=time()
    liste.append(b-a)
print(sum(liste)/100)

1.6449413061141969


### Page rank (unweighted)

In [10]:
liste=[]
for i in range(100):
    a=time()
    nx.pagerank(G,alpha=0.85,max_iter=20,weight=None)
    b=time()
    liste.append(b-a)
print(sum(liste)/100)

0.09266203880310059


### PageRank (weighted)

In [6]:
liste=[]
for i in range(100):
    a=time()
    nx.pagerank(G,alpha=0.85,max_iter=20,weight='weight')
    b=time()
    liste.append(b-a)
print(sum(liste)/100)


0.1326475238800049


### Label Propagation

In [7]:

liste=[]
for i in range(100):
    a=time()
    c=nx.algorithms.community.label_propagation.label_propagation_communities(G)
    b=time() 
    liste.append(b-a)
print(sum(liste)/100)


1.189708709716797e-06



### Minimum Spanning Tree


In [8]:

liste=[]
for i in range(100):
    a=time()
    nx.minimum_spanning_tree(G)
    b=time() 
    liste.append(b-a)
print(sum(liste)/100)
 

0.008865909576416016


### BFS

In [9]:
liste=[]
for i in range(100):
    a=time()
    t= nx.bfs_edges(G,"Jon-Snow",depth_limit=5)
    b=time() 
    liste.append(b-a)
print(sum(liste)/100)
print(liste[:5])

3.1709671020507814e-07
[1.9073486328125e-06, 1.1920928955078125e-06, 4.76837158203125e-07, 2.384185791015625e-07, 2.384185791015625e-07]


## Tests Neo4j (ms)
This test were not really automated because I wanted to pass through the cache memory in order to have to really time of algorithms. Therefore I had to restart the server between each test. That is why there not has much data as networkx. However, I think this is enough to have a decent overview of the capacity of each one.

### Code used:
#### Graph in memory
```
call gds.graph.create("G","Character","Interacts",
{
        relationshipProperties: 'weight'
    })
```

#### Betweenness centrality
```
 CALL gds.alpha.betweenness.stream({
nodeQuery: 'MATCH (p) RETURN id(p) AS id',
  relationshipQuery: 'MATCH (p1)-[]-(p2) RETURN id(p1) AS source, id(p2) AS target'
})
YIELD nodeId,centrality
return gds.util.asNode(nodeId).name as user,centrality
order by centrality DESC limit 1
```
#### Page rank (unweighted)
```
CALL gds.pageRank.stream('G',{maxIterations:20, dampingFactor:0.85})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC, name ASC limit 1
```
#### Page rank (weighted)
```
CALL gds.pageRank.stream('got',{maxIterations:20, dampingFactor:0.85,relationshipWeightProperty: 'weight'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC, name ASC limit 1
```

####  Label Propagation
```
CALL gds.labelPropagation.stream('G'})
YIELD nodeId, communityId AS Community
RETURN gds.util.asNode(nodeId).name AS Name, Community
ORDER BY Community, Name limit 1 
```

#### BFS
```
MATCH (a:Character{name:'Jon Snow'})
WITH id(a) AS startNode
CALL gds.alpha.bfs.stream('G', {startNode: startNode,maxDepth :5})
YIELD path
UNWIND [ n in nodes(path) | n.name] AS name
RETURN name
ORDER BY name limit 1
```

#### MST
```
MATCH (n:Character {name: 'Jon Snow'})
CALL gds.alpha.spanningTree.minimum.write({
  nodeProjection: 'Character',
  relationshipProjection: {
  	Interacts: {
    	type :'Interacts',
   	 	properties: 'weight',
    	orientation: 'UNDIRECTED'
    	}
    },
    startNodeId: id(n),
    relationshipWeightProperty: 'weight',
    writeProperty: 'MINST',
    weightWriteProperty: 'writeCost'
})
YIELD createMillis, computeMillis, writeMillis, effectiveNodeCount
RETURN createMillis, computeMillis, writeMillis, effectiveNodeCount;
```

### Raw results: 
betweenness #1 (ms) : 354,90,67,76,59,85,55,61,84,58,69

page rank #1 (ms) : 327,202,166, crash

betweenness #2 (ms) : 288,95,85,61,65,69,68,69

page rank #2 (ms) : 294,191,146

page rank #3 (ms) : 451, 241, 154,

page rank (weighted) : 347,215, 122,123

page rank (weighted) : 343,201, 138,114
page rank (weighted) : 284,233, 125,110

label propagation #1 (ms) : 230, 89,88,68,47,62,47,43,47

label propagation #2 (ms) : 120, 62,51,33,27,38,30

label propagation #3 (ms) : 170, 74, 64,79,44,49,47,46,67,

BFS (graph in memory) : environ 37;38,7;38,5 seconds for  max depth=5

MST :454,42 

MST(with graph projection during the algo) : , 1100, 1255,

MST (others first iterations) :488,483, 583,

MST:  620,66,57,53,60,46

## Synthesis
Algorithm | Time Neo4j (s) | Time Networkx(s)
:-: | :-: | :-:
Betweenness centrality | 0,32 (0,08 after) | 1,6
Page rank (unweighted) | 0,36 (0,1 after) | 0,09
Page rank(weighted) | 0,36 (0,1 after) | 0,12
Label Propagation | 0,17 (0,05 after) | 1e-6  (1e-5 for the first one)
Breadth First Search | 38 | 3e-7 (2e-6 for the 2 first ones)
Minimum spanning tree | 0,52 (0,06 after)| 7e-3 




## Conclusion

It is hard to have a clear decision because there is huge gap for either software. Most of the time Networkx is far better than neo4j. But is some case (Betweenness and Page rank) Neo4j is better or  getting even better after the first iteration due to the use of a cache. We also can see that weighted or unweighted Page rank doesn't change anything for neo4j but changes a bit for networkx. (This precisition is lower for neo4j)
However, there is a huge gap between Neo4j and Network regarding to BFS's algorithm (1e8 order of magnitude...)
